In [1]:
import sys
sys.path.append('C:/Pradap/Research/Python-Packages/enrique/')

In [2]:
import sys
import magellan as mg
import pandas as pd
import numpy as np
import os
import subprocess
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from magellan.feature.extractfeatures import apply_feat_fns
#------------------ functions --------------------------------------------


def visualize_tree(tree, feature_names):
    """Create tree png using graphviz.

    Args
    ----
    tree -- scikit-learn DecsisionTree.
    feature_names -- list of feature names.
    """
    with open("dt.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "dt.dot", "-o", "dt.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Could not run dot, ie graphviz, to "
             "produce visualization")


def get_code(tree, feature_names, target_names,
             spacer_base="    "):
    """Produce psuedo-code for decision tree.

    Args
    ----
    tree -- scikit-leant DescisionTree.
    feature_names -- list of feature names.
    target_names -- list of target (class) names.
    spacer_base -- used for spacing code (default: "    ").

    Notes
    -----
    based on http://stackoverflow.com/a/30104792.
    """
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    code_list = []
    def recurse(left, right, threshold, features, node, depth):
        spacer = spacer_base * depth
        if (threshold[node] != -2):
            code_str =  spacer + "if ( " + features[node] + " <= " + \
                  str(threshold[node]) + " ):"
            code_list.append(code_str)
            # print(spacer + "if ( " + features[node] + " <= " + \
            #       str(threshold[node]) + " ):")

            code_str = spacer + spacer_base +  "print \'" +spacer  +"Condition " + features[node] + " <= " + str(threshold[node]) + \
                  " PASSED " + "(  value : \'  + str(" +  str(features[node])  + ") + \')\'"
            code_list.append(code_str)

            # print(spacer + spacer_base + "print \'" + features[node] + " <= " + str(threshold[node]) +
            #       " PASSED " + "(  value : \'  + str(" +  str(features[node])  + ") + \')\'")
            if left[node] != -1:
                    recurse(left, right, threshold, features,
                            left[node], depth+1)
            #print(spacer + "}\n" + spacer +"else:")
            code_str = spacer + "else:"
            code_list.append(code_str)
            # print(spacer + "else:")


            code_str = spacer  + spacer_base + "print \'" +spacer  +"Condition " + features[node] + " <= " + str(threshold[node]) + \
                  " FAILED " + "(  value : \'  + str(" +  str(features[node])  + ") + \')\'"
            code_list.append(code_str)
            # print(spacer + spacer_base + "print \'" + features[node] + " <= " + str(threshold[node]) +
            #       " FAILED " + "(  value : \'  + str(" +  str(features[node])  + ") + \')\'")


            if right[node] != -1:
                    recurse(left, right, threshold, features,
                            right[node], depth+1)
            # print(spacer + "}")
        else:
            target = value[node]
            for i, v in zip(np.nonzero(target)[1],
                            target[np.nonzero(target)]):
                target_name = target_names[i]
                target_count = int(v)
                # print(spacer + "return " + str(target_name) + \
                #       " ( " + str(target_count) + " examples )")
                code_str = spacer + "return " + str(target_name) + \
                      " #( " + str(target_count) + " examples )"
                code_list.append(code_str)
                # print(spacer + "return " + str(target_name) + \
                #       " #( " + str(target_count) + " examples )")


    recurse(left, right, threshold, features, 0, 0)
    return code_list


def debug_dt(t1, t2, feat_table, clf):
    code = get_code(dt.clf, list(feat_table['feature_name']), ['False', 'True'])
    feat_vals = apply_feat_fns(t1, t2, feat_table)
    code = get_dbg_fn(code)
    #print code
    d = {}
    d.update(feat_vals)
    exec code in d
    ret_val = d['debug_fn']()
    print "Tuples match status : " + str(ret_val)



def get_dbg_fn(code):
    spacer_basic = '    '
    c = "def debug_fn(): \n"
    upd_code = [spacer_basic + e + "\n" for e in code]
    c = c + ''.join(upd_code)
    return c

In [3]:
import sys
sys.path.append('C:/Pradap/Research/Python-Packages/enrique')

wal = mg.read_csv('../magellan/datasets/books/walmart.csv', dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},
                                        low_memory=False, key='id')


bwk = mg.read_csv('../magellan/datasets/books/bowker.csv', dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},
                      low_memory=False, key='id')

ab = mg.AttrEquivalenceBlocker()
candset = ab.block_tables(bwk, wal, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


feat_table = mg.get_features_for_blocking(bwk, wal)

sample_cset = mg.sample_one_table(candset, 30)

mg.init_jvm('C:\\Program Files\\Java\\jre7\\bin\\server\\jvm.dll')

from magellan.feature.simfunctions import lev


#label_cset = mg.label(sample_cset, "gold_label")
#label_cset.save_table('book_label.pkl')
label_cset = mg.load_table('../testcases/book_label.pkl')
label_cset.set_property('ltable', bwk)
label_cset.set_property('rtable', wal)

fv = mg.extract_feat_vecs(label_cset, feat_table=feat_table, attrs_after=['gold_label'])


dt = mg.DTMatcher(random_state=81)
dt.fit(table=fv, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold_label'], target_attr='gold_label')
cols = [c not in ['_id', 'ltable.id', 'rtable.id', 'gold_label'] for c in fv.columns]
feature_names = fv.columns[cols]
visualize_tree(dt.clf, feature_names)
#debug_dt(wal.ix[2], bwk.ix[3], feat_table, dt.clf)
t1 = wal.ix[3]
t2 = bwk.ix[3]

f = apply_feat_fns(t1, t2, feat_table)

['id_id_exm', 'id_id_anm', 'id_id_lev', 'title_title_jac_qgm_3_qgm_3', 'title_title_cos_dlm_dc0_dlm_dc0', 'title_title_mel', 'author_author_jac_qgm_3_qgm_3', 'author_author_cos_dlm_dc0_dlm_dc0', 'author_author_jac_dlm_dc0_dlm_dc0', 'author_author_mel', 'author_author_nmw', 'author_author_sw', 'author_author_swg', 'numAuthors_numAuthors_exm', 'numAuthors_numAuthors_anm', 'numAuthors_numAuthors_lev', 'binding_binding_jac_qgm_3_qgm_3', 'binding_binding_cos_dlm_dc0_dlm_dc0', 'binding_binding_jac_dlm_dc0_dlm_dc0', 'binding_binding_mel', 'binding_binding_nmw', 'binding_binding_sw', 'binding_binding_swg', 'publisher_publisher_jac_qgm_3_qgm_3', 'publisher_publisher_cos_dlm_dc0_dlm_dc0', 'publisher_publisher_jac_dlm_dc0_dlm_dc0', 'publisher_publisher_mel', 'publisher_publisher_nmw', 'publisher_publisher_sw', 'publisher_publisher_swg', 'isbn_isbn_lev', 'isbn_isbn_jar', 'isbn_isbn_jwn', 'isbn_isbn_sdx', 'isbn_isbn_exm', 'isbn_isbn_jac_qgm_3_qgm_3', 'pubYear_pubYear_exm', 'pubYear_pubYear_anm', 'p

In [4]:
visualize_tree(dt.clf, feature_names)

In [5]:
from sklearn.externals.six import StringIO

In [6]:
import pydotplus

In [7]:
dot_data = StringIO()

In [8]:
tree = dt.clf

In [9]:
from sklearn import tree

In [10]:
tree.export_graphviz(dt.clf, out_file=dot_data)

In [11]:
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())

In [13]:
graph.write_jpeg('pp.jpg')

InvocationException: GraphViz's executables not found

In [14]:
from subprocess import call

In [16]:
import os

In [19]:
os.environ['COMSPEC']

'C:\\WINDOWS\\system32\\cmd.exe'